In [ ]:
# Check on expected morning azimuth limits
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import EarthLocation, get_sun, AltAz
import rubin_nights.dayobs_utils as rn_dayobs

day_obs = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
_, dawn = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=0)
end = rn_dayobs.day_obs_to_time(day_obs) + TimeDelta(1.3, 'jd')
end, _ = rn_dayobs.day_obs_sunset_sunrise(rn_dayobs.time_to_day_obs(rn_dayobs.day_obs_to_time(day_obs) + TimeDelta(1, 'jd')), sun_alt=0)
times = Time(np.arange(dawn.mjd, end.mjd, 1/24), format='mjd', scale='utc')
location = EarthLocation.of_site("Rubin")
sun_pos = get_sun(times)
altazframes = AltAz(location=location, obstime=times)
altaz = sun_pos.transform_to(altazframes)

theta = altaz.az.rad
r = 90 - altaz.alt.deg 

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, r, 'k.')
ax.plot(theta[0], r[0], 'yo')
# extrapolated limits .. add to safety masks
ax.plot([theta[0]+np.radians(45), theta[0]+np.radians(45)], [0, 90], color='r')
ax.plot([theta[-1]-np.radians(10), theta[-1]-np.radians(10)], [0, 90], color='r')

# Summit operational constraints (#summit-announce, 2025-10-27)
ax.plot(np.radians([270, 270]), [0, 90], color='b')
ax.plot(np.radians([315, 315]), [0, 90], color='b')

ax.set_theta_zero_location('N')  # Set 0 degrees (azimuth) to North
ax.set_theta_direction(-1)       # Set direction to clockwise

print(f"Dayobs {day_obs}")
print(f"Sunrise az {altaz.az.deg[0] :.0f}, Ending az {altaz.az.deg[-1]:.0f}")
print(f"Limits: {altaz.az.deg[0] + 45 :.0f} {altaz.az.deg[-1] - 10:.0f}")

print("summit limits 270 to 315")